# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import os
import sys
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import string
nltk.download(['punkt', 'wordnet', 'stopwords'])
warnings.simplefilter('ignore')

/Users/hanz/anaconda3/envs/udacity_env/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/hanz/anaconda3/envs/udacity_env/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; f

[nltk_data] Downloading package punkt to /Users/hanz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hanz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hanz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disasterData.db')


In [3]:
df = pd.read_sql_table("disasterData", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.related.unique()

array([1, 0])

In [5]:
df.replace(2,1,inplace=True)

In [6]:
X = df["message"].values
Y = df.drop(columns=["id","message","original","genre"],axis=1).values

In [7]:
df.columns[4:]

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [8]:
X[:10]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I would like to receive the messages, thank you',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'],
      dtype=object)

In [9]:
Y[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    clean_tokens=[]
    lemmatizer = WordNetLemmatizer()
    lower_text=text.lower()
    p=re.compile("["+re.escape(string.punctuation)+"]")
    normalized_text=p.sub("",lower_text)
    word_tokens=word_tokenize(normalized_text)
    # Remove stop words
    stop_words = stopwords.words('english')
    filtered_words = [w for w in word_tokens if not w in stop_words]
    for tok in filtered_words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [11]:
def tokenize_2(text):
    '''
    tokenization, lemmatization of the raw text
    INPUT:
        raw text
    OUTPUT:
        cleaned tokens
    '''
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [12]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [13]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline_simple = Pipeline([
    ("vect",CountVectorizer(tokenizer = tokenize)),
    ("tfidf",TfidfTransformer()),
    ("clf",MultiOutputClassifier(RandomForestClassifier()))
    #("clf",MultiOutputClassifier(KNeighborsClassifier()))
])


In [15]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [16]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline_simple.fit(X_train, y_train)

In [ ]:
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [ ]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])

In [ ]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f9e1c8dc488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
        verbose=0),
             n_jobs=1),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dty

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
target_labels=list(df.columns[4:])
y_test_pred = pipeline_simple.predict(X_test)
print(classification_report(y_test,y_test_pred,target_names=target_labels))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.89      5035
               request       0.78      0.44      0.57      1120
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.57      0.65      2773
          medical_help       0.59      0.07      0.12       493
      medical_products       0.74      0.08      0.14       335
     search_and_rescue       0.33      0.01      0.02       190
              security       0.00      0.00      0.00       116
              military       0.66      0.08      0.15       230
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.33      0.47       408
                  food       0.83      0.52      0.64       729
               shelter       0.85      0.34      0.49       590
              clothing       0.50      0.02      0.04        85
                 money       0.69      

In [ ]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_test)
type_of_target(y_test_pred)

'multilabel-indicator'

In [ ]:
target_labels=list(df.columns[4:])
y1_test_pred = pipeline1.predict(X_test)
print(classification_report(y_test,y1_test_pred,target_names=target_labels))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      5035
               request       0.77      0.50      0.61      1120
                 offer       0.00      0.00      0.00        29
           aid_related       0.78      0.61      0.69      2773
          medical_help       0.62      0.28      0.39       493
      medical_products       0.68      0.30      0.42       335
     search_and_rescue       0.67      0.17      0.28       190
              security       0.30      0.08      0.12       116
              military       0.65      0.33      0.43       230
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.60      0.66       408
                  food       0.80      0.71      0.75       729
               shelter       0.77      0.51      0.61       590
              clothing       0.68      0.47      0.56        85
                 money       0.51      

In [ ]:
target_labels=list(df.columns[4:])
y2_test_pred = pipeline2.predict(X_test)
print(classification_report(y_test,y2_test_pred,target_names=target_labels))

                        precision    recall  f1-score   support

               related       0.86      0.92      0.89      5035
               request       0.75      0.62      0.68      1120
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.69      0.71      2773
          medical_help       0.59      0.31      0.41       493
      medical_products       0.68      0.29      0.40       335
     search_and_rescue       0.69      0.15      0.25       190
              security       0.00      0.00      0.00       116
              military       0.69      0.28      0.40       230
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.57      0.64       408
                  food       0.82      0.67      0.74       729
               shelter       0.76      0.54      0.63       590
              clothing       0.72      0.36      0.48        85
                 money       0.51      

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline_simple.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f9e1c8dc488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [ ]:
pipeline1.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__count_vectorizer', 'features__text_pipeline__tfidf_transformer', 'features__text_pipeline__count_vectorizer__analyzer', 'features__text_pipeline__count_vectorizer__binary', 'features__text_pipeline__count_vectorizer__decode_error', 'features__text_pipeline__count_vectorizer__dtype', 'features__text_pipeline__count_vectorizer__encoding', 'features__text_pipeline__count_vectorizer__input', 'features__text_pipeline__count_vectorizer__lowercase', 'features__text_pipeline__count_vectorizer__max_df', 'features__text_pipeline__count_vectorizer__max_features', 'features__text_pipeline__count_vectorizer__min_df', 'features__text_pipeline__count_vectorizer__ngram_range', 'features__text_pipeline__count_vectorizer__preprocessor', 'features_

In [ ]:
print(pipeline2.get_params().keys())

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__estimator__C', 'clf__estimator__estimator__class_weight', 'clf__estimator__estimator__dual', 'clf__estimator__estimator__fit_intercept', 'clf__estimator__estimator__intercept_scaling', 'clf__estimator__estimator__loss', 'clf__estimator__estimator__max_iter', 'clf__estimator__estimator__multi_class', 'clf__estimator__estimator__penalty', 'clf__estimator__estimator__random_state', 'clf__estimator__estimator__tol', 'clf__estimator__estimator__verbose', 'clf__estimator__estimator', 'clf__estimator__n_jobs', 'clf__estimator', 

In [ ]:
pipeline_simple.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [ ]:
parameters = {
    #'clf__estimator__n_estimators': [20,50,80],
    'clf__estimator__min_samples_leaf': [5, 10, 20],
    'clf__estimator__max_features': [0.5,1,"log2"],
    #'vect__max_df':[1, 5,10]
}

cv = GridSearchCV(pipeline_simple, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test, y_pred,target_names=target_labels))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      5035
               request       0.75      0.54      0.63      1120
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.64      0.69      2773
          medical_help       0.57      0.27      0.36       493
      medical_products       0.78      0.32      0.45       335
     search_and_rescue       0.77      0.23      0.35       190
              security       0.00      0.00      0.00       116
              military       0.62      0.33      0.44       230
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.61      0.67       408
                  food       0.80      0.78      0.79       729
               shelter       0.75      0.60      0.67       590
              clothing       0.75      0.56      0.64        85
                 money       0.65      

In [ ]:
cv.best_params_

{'clf__estimator__max_features': 0.5, 'clf__estimator__min_samples_leaf': 10}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open('model_pipeline2.pkl', 'wb') as model_file:
  pickle.dump(pipeline2, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.